In [1]:
import numpy as np
import pywt

import seaborn as sns #绘制confusion matrix heatmap

import os
import scipy.io as sio

from statsmodels.tsa.ar_model import AR
from sklearn.mixture import GaussianMixture

import tqdm
import  time

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os


os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

KTF.set_session(sess)

Using TensorFlow backend.
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type

In [3]:
import warnings
warnings.simplefilter('ignore') #忽略警告

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import xgboost

In [4]:
sample_rate = 256
origin_channel = 16


SAMPLE_CHANNEL = ['Pz' , 'PO3' , 'PO4' , 'O1' , 'O2' , 'Oz' , 'O9' , 'FP2' ,
                  'C4' , 'C6' , 'CP3' , 'CP1' ,
                  'CPZ' , 'CP2' , 'CP4' , 'PO8']

LABEL2STR = {0:'sen' , 1:'hong' , 2:'zhao',
             3:'fen' , 4:'xiao' , 5:'yu' , 
             6:'bin' , 7:'wang' , 8:'wei' , 
             9:'fei'}

CLIP_FORWARD = 1 #首部裁掉时间
CLIP_BACKWARD = 1 #尾部裁掉时间

trial_time = 3 #segment second


#是否进行归一化
#reference:a study on performance increasing in ssvep based bci application
#IS_NORMALIZE = True

#是否进行滤波
#IS_FILTER = False
#EEG频率范围
#reference:a study on performance increasing in ssvep based bci application
LO_FREQ = 0.5
HI_FREQ = 40

#是否陷波
#IS_NOTCH = False
NOTCH_FREQ = 50 #陷波 工频

from keras.utils import to_categorical

In [5]:
def load_data(filename):

    data = sio.loadmat(file_name=filename)['data_received'] #length*16 matrix

    data = data[CLIP_FORWARD * sample_rate : - CLIP_BACKWARD * sample_rate] #首部 尾部 进行裁剪
   
    return data 

In [6]:
def separate(data , label , overlap_length):
    train_data = []
    train_labels = []

    size = sample_rate * trial_time #一小段 256*3 个数据点
    data_length = data.shape[0]

    idx = 0
    
    while idx<=data_length-size:
        train_data.append(data[idx : idx+size , :])
        train_labels.append(label)

        idx = idx + (size - overlap_length)
        
    return np.array(train_data) , np.array(train_labels)

In [7]:
def shuffle_t_v(filenames):
    # np.random.shuffle(filenames)
    
    return np.random.choice(filenames , size=10) #20次的计算准确率中 每次随机选择10个样本进行训练测试

def combine(freq):    
    overlap_length = 2*256 #重叠2秒数据
    
    #保证随机性 进行置乱
    person_0_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/0/' % freq) )
    person_1_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/1/' % freq) )
    person_2_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/2/' % freq) )
    person_3_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/3/' % freq) )
    person_4_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/4/' % freq) )
    person_5_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/5/' % freq) )
    person_6_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/6/' % freq) )
    person_7_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/7/' % freq) )
    person_8_filenames = shuffle_t_v( os.listdir('../incremental/data/base_rf/%s/8/' % freq) )

    #打开信号文件 并 合并
    person_0 = np.concatenate([load_data('../incremental/data/base_rf/%s/0/' % freq + filename) for filename in person_0_filenames] , axis = 0)
    person_1 = np.concatenate([load_data('../incremental/data/base_rf/%s/1/' % freq + filename) for filename in person_1_filenames] , axis = 0)
    person_2 = np.concatenate([load_data('../incremental/data/base_rf/%s/2/' % freq + filename) for filename in person_2_filenames] , axis = 0)
    person_3 = np.concatenate([load_data('../incremental/data/base_rf/%s/3/' % freq + filename) for filename in person_3_filenames] , axis = 0)
    person_4 = np.concatenate([load_data('../incremental/data/base_rf/%s/4/' % freq + filename) for filename in person_4_filenames] , axis = 0)
    person_5 = np.concatenate([load_data('../incremental/data/base_rf/%s/5/' % freq + filename) for filename in person_5_filenames] , axis = 0)
    person_6 = np.concatenate([load_data('../incremental/data/base_rf/%s/6/' % freq + filename) for filename in person_6_filenames] , axis = 0)
    person_7 = np.concatenate([load_data('../incremental/data/base_rf/%s/7/' % freq + filename) for filename in person_7_filenames] , axis = 0)
    person_8 = np.concatenate([load_data('../incremental/data/base_rf/%s/8/' % freq + filename) for filename in person_8_filenames] , axis = 0)
    
    #============
    #训练数据分段
    train_person_data_0 , train_person_labels_0 = separate(person_0 , label = 0 , overlap_length=overlap_length)
    train_person_data_1 , train_person_labels_1 = separate(person_1 , label = 1 , overlap_length=overlap_length)
    train_person_data_2 , train_person_labels_2 = separate(person_2 , label = 2 , overlap_length=overlap_length)
    train_person_data_3 , train_person_labels_3 = separate(person_3 , label = 3 , overlap_length=overlap_length)
    train_person_data_4 , train_person_labels_4 = separate(person_4 , label = 4 , overlap_length=overlap_length)
    train_person_data_5 , train_person_labels_5 = separate(person_5 , label = 5 , overlap_length=overlap_length)
    train_person_data_6 , train_person_labels_6 = separate(person_6 , label = 6 , overlap_length=overlap_length)
    train_person_data_7 , train_person_labels_7 = separate(person_7 , label = 7 , overlap_length=overlap_length)
    train_person_data_8 , train_person_labels_8 = separate(person_8 , label = 8 , overlap_length=overlap_length)

    #合并数据
    train_data = np.concatenate((train_person_data_0 , train_person_data_1 , train_person_data_2 ,
                                 train_person_data_3 , train_person_data_4 , train_person_data_5 ,
                                 train_person_data_6 , train_person_data_7 , train_person_data_8 ,
                                 ))
    
    train_labels = np.concatenate((train_person_labels_0 , train_person_labels_1 , train_person_labels_2 ,
                                   train_person_labels_3 , train_person_labels_4 , train_person_labels_5 ,
                                   train_person_labels_6 , train_person_labels_7 , train_person_labels_8 ,
                                    ))
    
    #产生索引并置乱
    idx_train_data = list(range(train_data.shape[0]))
    np.random.shuffle(idx_train_data)

    #将训练数据置乱
    train_data = train_data[idx_train_data]
    train_labels = train_labels[idx_train_data]
        
    return train_data , train_labels


In [8]:
def session_data_labels(session_id , freq , is_training):
    if is_training:
        overlap_length = 256*2
    else:
        overlap_length = 0
    
    str_freq = str(freq)
    
    subjcets = os.listdir('../incremental/data/incremental/%s/s%d/' % (str_freq , session_id)) #受试者ID
    
    data = []
    labels = []
    
    for subjcet in subjcets:
        filenames = os.listdir('../incremental/data/incremental/%s/s%d/%s/' % (str_freq , session_id , subjcet))
        
        person = np.concatenate([load_data('../incremental/data/incremental/%s/s%d/%s/%s' % (str_freq , session_id , subjcet , filename)) for filename in filenames] , axis = 0)
        
        person_data , person_label = separate( person , label = int(subjcet) , overlap_length = overlap_length)
        
        data.append(person_data)
        labels.append(person_label)
    
    #合并数据
    data = np.concatenate(data)
    labels = np.concatenate(labels)
    
    #shuffle
    idx_data = list(range(data.shape[0]))
    np.random.shuffle(idx_data)

    data = data[idx_data]
    labels = labels[idx_data]
    
    return data , labels


In [9]:
def get_center(data , label):
    centers = []
    
    for label_id in range(9): #一共9个受试者
        equal_idx = np.where(label == label_id)
    
        center = np.mean(data[equal_idx] , axis = 0)
        centers.append(center)
        
    return np.array(centers)

def get_center_simple(data):
    '''
    计算单个用户的脑电的中心
    '''
    return np.mean(data , axis=0)

# PCA降维

## kmeans聚类

In [10]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input

In [11]:
encoder_input = Input(shape = (256*3*16 , ))

#encoder
encoder = Dense(units=512 , activation='relu')(encoder_input)
encoder = Dense(units=256 , activation='relu')(encoder)
encoder = Dense(units=128 , activation='relu')(encoder)
encoder_output = Dense(units=64 , activation='relu')(encoder) #聚类需要使用的2维特征

#decoder
decoder = Dense(units=128 , activation='relu')(encoder_output)
decoder = Dense(units=256 , activation='relu')(decoder)
decoder = Dense(units=512 , activation='relu')(decoder)
decoder_output = Dense(units=256*3*16 , activation='relu')(decoder)


In [12]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

### 6Hz

In [12]:
all_centers = []
all_time = []

for freq in [6]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    


freq :  6
session :  1 [[ 841.03406     15.79293  ]
 [-226.33699    -29.086267 ]
 [ 389.8304     -15.756354 ]
 [-164.90143    176.24994  ]
 [ 263.31952    -24.09588  ]
 [ 622.456        2.302034 ]
 [ 115.28172    -26.212023 ]
 [ -60.116352   -44.424652 ]
 [ 514.0633      -1.5250419]] 0.1972731
session :  3 [[-213.88426    88.404755]
 [ 120.24876  -141.99725 ]
 [ 413.03708   188.22801 ]
 [ -15.129155  -48.57789 ]
 [-126.213005   24.425646]
 [-242.94357   112.783554]
 [-283.1882    141.75974 ]
 [ -53.943424  -23.17649 ]
 [  99.13313  -126.473885]] 0.06044759999999627
session :  5 [[-110.49078   -14.838325]
 [ 284.11444   -32.502155]
 [  47.38447   172.26665 ]
 [ 131.42108   -32.64022 ]
 [   5.31623   -27.088453]
 [ 184.41055   -34.666412]
 [ -73.920654  -21.691206]
 [ 317.564     -33.36165 ]
 [  71.14902   -30.842947]] 0.06862360000000933
session :  6 [[ -26.776672  -66.38209 ]
 [1149.6708   -156.49503 ]
 [-634.7317    -16.825886]
 [ 345.0003    515.30505 ]
 [ 268.42508   -89.23302 ]
 [ 

In [12]:
all_centers = []
all_time = []

for freq in [7.5]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    


freq :  7.5
session :  1 [[ -9.87312     3.6735303]
 [ 47.222473    9.9345875]
 [ -6.0012875 -32.251255 ]
 [-41.74517    50.53848  ]
 [ 11.345608   -1.368478 ]
 [-17.96083   -17.609072 ]
 [-24.760798    9.858317 ]
 [-43.98355    33.16809  ]
 [ 58.452827   21.445736 ]] 0.0650976
session :  3 [[-17.886723   -3.4190795]
 [128.0091     29.199171 ]
 [-50.723637   81.34667  ]
 [-10.601768  -28.8022   ]
 [ -1.5789691  -4.5052214]
 [-18.321884  -18.108255 ]
 [  7.612451  -36.533176 ]
 [ -2.043262  -17.930624 ]
 [115.469986   26.198242 ]] 0.06533890000000042
session :  5 [[-11.986294  -21.178892 ]
 [-81.10913    97.48542  ]
 [129.41078    49.77595  ]
 [  8.5024    -18.102467 ]
 [ -7.3710027 -26.400063 ]
 [-18.434107  -13.721245 ]
 [  3.584116  -23.220802 ]
 [-10.763332  -28.828398 ]
 [137.37936    54.337406 ]] 0.059540400000003046
session :  6 [[-34.49383   -21.030918 ]
 [431.41983    -2.7935557]
 [-26.628895   70.94907  ]
 [-27.152025   10.941036 ]
 [-22.708525   97.58728  ]
 [-32.84185   -15.

In [12]:
all_centers = []
all_time = []

for freq in [8.5]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    


freq :  8.5
session :  1 [[ 4.16626091e+01 -7.27419043e+00]
 [-8.81259060e+00  4.77106512e-01]
 [-4.13709307e+00  2.05852242e+01]
 [-8.92675114e+00 -1.10410709e+01]
 [ 2.79053354e+00  7.46134233e+00]
 [ 4.92634048e+01 -7.32707644e+00]
 [-1.16728924e-01  3.16882057e+01]
 [-3.27077881e-02  1.99262600e+01]
 [ 1.11880831e-01 -7.77108788e-01]] 0.0693359
session :  3 [[-1.7269699e+01 -6.8698297e+00]
 [ 8.1792099e+01 -2.1791716e+01]
 [ 2.2966099e+01  5.7645691e+01]
 [-7.7477722e+00 -5.2201295e+00]
 [-1.3762439e+01  3.5473749e-02]
 [ 2.8330486e+01  6.4688774e+01]
 [-1.8493389e+01 -2.8493943e+00]
 [ 1.7840443e+01  5.0975918e+01]
 [-1.1953937e+01 -5.9328456e+00]] 0.05515640000000133
session :  5 [[  3.85666   -19.325634 ]
 [-47.30778    57.93997  ]
 [ 82.81519    31.265965 ]
 [-12.294344   -8.604472 ]
 [ -5.4738417 -14.84148  ]
 [ 14.027012  -14.679574 ]
 [-17.731094    1.5612825]
 [ -8.20499   -13.379711 ]
 [  9.229276  -16.37672  ]] 0.056698700000012536
session :  6 [[-43.11992   -19.628616 ]


In [12]:
all_centers = []
all_time = []

for freq in [10]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_pca)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    


freq :  10
session :  1 [[ -9.068515   23.246708 ]
 [ 41.252605  -65.45651  ]
 [ 91.94276    40.939285 ]
 [-27.240955    4.447935 ]
 [ -5.7323856  -8.333161 ]
 [ 82.01645    59.770477 ]
 [ 98.73237    16.729208 ]
 [-22.51091    14.109867 ]
 [-12.545635  -12.969641 ]] 0.0776471
session :  3 [[ -7.7918396 -30.853592 ]
 [ 96.57647    22.310091 ]
 [-61.742996   97.92483  ]
 [-36.985992   10.63233  ]
 [-21.580244  -27.72118  ]
 [-28.318375   -9.148458 ]
 [-66.517105   78.29668  ]
 [-16.50211   -36.615738 ]
 [ 88.72163    17.309286 ]] 0.06340709999999916
session :  5 [[-20.02621    -6.931774 ]
 [112.02672   -53.073933 ]
 [ 62.677586   86.90036  ]
 [-25.104948   -5.0101   ]
 [-26.648825   -7.5736103]
 [-22.373419    1.7193458]
 [-28.882584   -2.8150296]
 [-16.234047   -6.357594 ]
 [ 60.232998   84.247856 ]] 0.05558020000000852
session :  6 [[-26.85943     -7.810604  ]
 [117.498344   -24.803093  ]
 [ 36.356064    82.84787   ]
 [ 18.29755    -29.047634  ]
 [-33.809013     0.82097554]
 [134.0252

## DBSCAN聚类

## GMM聚类

In [14]:
all_centers = []
all_time = []

for freq in [6]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , _centers , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    


freq :  6
session :  1 [[-12.959373  -11.963699 ]
 [ 94.549934   20.699448 ]
 [-40.848454   34.920036 ]
 [  9.242683   14.724361 ]
 [ -4.6711955 -44.606678 ]
 [  1.2790338  -6.2808604]
 [-22.9298      9.430716 ]
 [  5.6024137 -23.827173 ]
 [-55.792534   63.31543  ]] 0.029536800000002472
session :  3 [[-16.535654  32.77816 ]
 [204.71147   -8.64754 ]
 [-36.497704 -36.34811 ]
 [-21.363096  16.501112]
 [-24.8301     3.629301]
 [-32.207108 -23.15803 ]
 [-14.362703  39.644543]
 [-37.19156  -39.953922]
 [-13.296611  43.281296]] 0.03622980000000098
session :  5 [[-25.97834     -6.0256996 ]
 [ 80.20225     -4.2085986 ]
 [ -2.414296    62.51113   ]
 [ 23.56706     -9.9253025 ]
 [  3.9958465  -12.173977  ]
 [-20.846756   -10.971154  ]
 [ 89.92567     -3.339596  ]
 [-25.091194    -0.09492765]
 [-13.212736   -13.420394  ]] 0.06576610000001892
session :  6 [[-24.763597    77.41472   ]
 [296.8975       0.38494703]
 [-40.305088   -10.085411  ]
 [-49.317226    27.13761   ]
 [-21.227772   -38.782265  ]


In [13]:
all_centers = []
all_time = []

for freq in [7.5]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , _centers , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    


freq :  7.5
session :  1 [[ -8.512972   20.169722 ]
 [-19.317677   -8.179623 ]
 [ 80.95834   -33.265095 ]
 [ 24.248981   68.90833  ]
 [  1.4180418  -7.4746504]
 [-26.096544  -17.230473 ]
 [ 58.451195  -39.567272 ]
 [-19.591278    9.757782 ]
 [100.65973   -26.222055 ]] 0.0406468
session :  3 [[-32.92987    -25.939316  ]
 [ 19.758291    40.769016  ]
 [-18.683674     8.684259  ]
 [ 75.465996     6.4753866 ]
 [ 21.194063   -39.447723  ]
 [ 40.425728   -36.163223  ]
 [ -0.65537703 -26.642065  ]
 [-23.507713     0.72828394]
 [  9.707067    43.788597  ]] 0.030296000000006984
session :  5 [[-15.331266  -17.487284 ]
 [107.28162    10.694158 ]
 [-31.389515   66.86453  ]
 [ 13.511596   -3.7105215]
 [-23.57242     2.685237 ]
 [-21.96815   -13.344099 ]
 [  3.0172899 -11.041758 ]
 [ -7.941036  -18.044561 ]
 [  9.852599   -4.22173  ]] 0.029834999999991396
session :  6 [[-2.52195873e+01 -6.34383559e-01]
 [ 3.93769562e+02  5.78129828e-01]
 [-3.06575012e+01  7.08211823e+01]
 [-2.90346165e+01 -1.52022295

In [13]:
all_centers = []
all_time = []

for freq in [8.5]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , _centers , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    


freq :  8.5
session :  1 [[-16.30189   -16.760923 ]
 [ 84.87126   -12.255501 ]
 [  2.63058    24.306185 ]
 [-13.229313   33.13599  ]
 [ -6.3986206  -5.034193 ]
 [-19.411097  -18.509659 ]
 [ -8.934164    9.43967  ]
 [ 10.621984   39.409756 ]
 [-13.151014   40.87099  ]] 0.0657273
session :  3 [[ 14.421317  -27.402388 ]
 [129.43494     9.831774 ]
 [-16.458921  -13.901213 ]
 [-26.982023   39.90481  ]
 [-25.875933   27.039536 ]
 [-20.515743   -0.6174519]
 [-22.674042  -11.931215 ]
 [-15.074618   -6.9005327]
 [-25.422686   21.809963 ]] 0.019254300000000057
session :  5 [[ -5.004483    4.626053 ]
 [138.19461   -23.194399 ]
 [  5.8444943  90.60906  ]
 [-28.292334  -15.62281  ]
 [  5.6041484  14.781805 ]
 [-33.653484  -24.83967  ]
 [-23.787916   -9.487069 ]
 [-21.933592  -13.613951 ]
 [-29.216417   -9.843738 ]] 0.017468600000000833
session :  6 [[-50.555244   12.83051  ]
 [ 90.46869   -72.19419  ]
 [116.832344   72.35397  ]
 [  1.1464337 -28.558325 ]
 [-35.486473    4.3531356]
 [ -5.0493584 -14

In [13]:
all_centers = []
all_time = []

for freq in [10]:
    sub_centers = []
    sub_time = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        pca = PCA(n_components=2)
        start = time.clock()
        X_pca = pca.fit_transform(data_feature)
        time1 = time.clock() - start
        
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('session : ' , session_id , _centers , time1 + time2)
    
    all_centers.append(sub_centers)
    all_time.append(sub_time)
    

freq :  10
session :  1 [[-13.882088   -18.793104  ]
 [142.76842    -14.522353  ]
 [ -8.683321    91.89341   ]
 [-13.339914    -4.1472    ]
 [ -3.320083   -13.755402  ]
 [-22.730019   -16.889011  ]
 [-21.352863     0.37057656]
 [130.75368      5.338324  ]
 [123.78542    -16.463984  ]] 0.0240346
session :  3 [[-10.237656  -19.52956  ]
 [ 78.91703    51.901875 ]
 [-71.39296    48.489483 ]
 [ 19.312479  -19.731384 ]
 [  0.7686747 -33.35722  ]
 [-53.855595   28.84251  ]
 [106.41902    70.90032  ]
 [ 20.36254    -4.3387566]
 [  4.3959527 -26.491623 ]] 0.017293900000005635
session :  5 [[-20.379173   -7.413877 ]
 [ 93.4192    -22.966402 ]
 [ 27.30985    55.38455  ]
 [-10.240775    1.3109554]
 [-15.208121   -6.408049 ]
 [ -8.4163     -1.3196698]
 [-14.916953   -1.1568015]
 [-20.760885   -4.6619368]
 [-23.090208   -6.1635685]] 0.022737000000006446
session :  6 [[ 1.1521382e+02 -4.0865746e+01]
 [-3.1899801e+01  4.3924496e-02]
 [ 5.6713497e+01  6.8642464e+01]
 [ 3.4145088e+00 -2.9257061e+01]
 [ 

# LDA降维

## kmeans and gmm聚类

In [19]:
from keras.models import Sequential , Model
from keras.layers import Dense , Dropout , Conv2D , MaxPooling2D , Reshape , BatchNormalization , Flatten
from keras.layers import Input

In [20]:
encoder_input = Input(shape = (256*3*16 , ))

#encoder
encoder = Dense(units=512 , activation='relu')(encoder_input)
encoder = Dense(units=256 , activation='relu')(encoder)
encoder = Dense(units=128 , activation='relu')(encoder)
encoder_output = Dense(units=64 , activation='relu')(encoder) #聚类需要使用的2维特征

#decoder
decoder = Dense(units=128 , activation='relu')(encoder_output)
decoder = Dense(units=256 , activation='relu')(decoder)
decoder = Dense(units=512 , activation='relu')(decoder)
decoder_output = Dense(units=256*3*16 , activation='relu')(decoder)


In [21]:
autoencoder = Model(inputs=encoder_input , outputs=decoder_output)
encoder = Model(inputs=encoder_input , outputs=encoder_output)

In [22]:
for freq in [6]:
    sub_centers = []
    sub_time = []
    
    sub_centers_gmm = []
    sub_time_gmm = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        '''
        kmeans
        '''
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_lda)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('kmeans session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
        
        '''
        gmm
        '''
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('gmm session : ' , session_id , _centers , time1 + time2)
    


freq :  6


InvalidArgumentError: You must feed a value for placeholder tensor 'dense_8_sample_weights' with dtype float and shape [?]
	 [[Node: dense_8_sample_weights = Placeholder[dtype=DT_FLOAT, shape=[?], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[Node: loss_1/mul/_1777 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_925_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
for freq in [7.5]:
    sub_centers = []
    sub_time = []
    
    sub_centers_gmm = []
    sub_time_gmm = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        '''
        kmeans
        '''
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_lda)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('kmeans session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
        
        '''
        gmm
        '''
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('gmm session : ' , session_id , _centers , time1 + time2)
    


In [ ]:
for freq in [8.5]:
    sub_centers = []
    sub_time = []
    
    sub_centers_gmm = []
    sub_time_gmm = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        '''
        kmeans
        '''
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_lda)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('kmeans session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
        
        '''
        gmm
        '''
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('gmm session : ' , session_id , _centers , time1 + time2)
    


In [ ]:
for freq in [10]:
    sub_centers = []
    sub_time = []
    
    sub_centers_gmm = []
    sub_time_gmm = []
    
    print('freq : ' , freq)
    
    for session_id in [1 , 3 , 5 , 6 , 7 , 8 , 9 , 11 , 12 , 13]:
        
        data , labels = session_data_labels(session_id , freq , is_training=True)
        data = np.reshape(data , newshape=(-1 , 768*16))
        data = (data-np.mean(data)) / np.std(data)
        
        #自编码器的训练
        autoencoder.compile(optimizer='adam' , loss = 'mse')
        autoencoder.fit(data , data , batch_size=32 , epochs=50 , verbose=0)
        #encoder部分进行预测输出
        data_feature = encoder.predict(x = data) #输出2维新特征 在xOy坐标系绘制
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        start = time.clock()
        X_lda = lda.fit_transform(data_feature , labels)
        time1 = time.clock() - start
        
        '''
        kmeans
        '''
        kmeans = KMeans(n_clusters=9)
        start = time.clock()
        _ = kmeans.fit_transform(X_lda)
        time2 = time.clock() - start
        
        sub_centers.append(kmeans.cluster_centers_)
        sub_time.append(time1 + time2)
        
        print('kmeans session : ' , session_id , kmeans.cluster_centers_ , time1 + time2)
        
        '''
        gmm
        '''
        gmm = GaussianMixture(n_components=9)
        start = time.clock()
        res = gmm.fit_predict(X_pca)
        time2 = time.clock() - start
        
        _centers = get_center(X_pca , res)
        
        sub_centers.append(_centers)
        sub_time.append(time1 + time2)
        
        print('gmm session : ' , session_id , _centers , time1 + time2)
    


## DBSCAN聚类

## GMM聚类